In [5]:
import ee
import geemap
import os

# Authenticate to Earth Engine
try:
  ee.Initialize(project='envr451-2024')
except Exception as e:
  ee.Authenticate()
  ee.Initialize(project='envr451-2024')

# Planet
Planet provides us with 4.77m resolution all the way to Jan 2016.
https://developers.google.com/earth-engine/datasets/catalog/projects_planet-nicfi_assets_basemaps_americas

Note - with the NICFI project, we can only get biannual images up until 2020; in 2020 we get
planet_medres_normalized_analytic_2020-06_2020-08_mosaic
planet_medres_normalized_analytic_2020-09_mosaic
planet_medres_normalized_analytic_2020-10_mosaic
planet_medres_normalized_analytic_2020-11_mosaic
planet_medres_normalized_analytic_2020-12_mo

Here, I am considering medians for all years. Consider, however, that some images have clouds, and we expect worse quality for the images from 2015-2020 since 2015-2019 only has 2 images per year, and 2020 has 5, while for 2021-2024 we get all 12 images for the year.

This can be improved once we get all images from McGill. Also, these should be at 3.7m resolution.saic

In [260]:
piura_shp = ee.FeatureCollection('projects/envr451-2024/assets/piura').geometry()
basemap = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/americas').filter(ee.Filter.date('2015-01-01', '2024-12-31')) \
            .map(lambda image: image.clip(piura_shp))

# Define the years for which you want to calculate geometric medians.
years = ee.List.sequence(2015, 2024)

# Function to calculate geometric median for each year.
def calculate_median(year):
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = ee.Date.fromYMD(year, 12, 31)
    yearly_collection = basemap.filterDate(start_date, end_date)
    median = yearly_collection.reduce(ee.Reducer.median())
    return median.set('year', year)

# Map over the list of years and calculate geometric medians.
yearly_medians = ee.ImageCollection(years.map(calculate_median)).map(lambda image: image.clip(piura_shp))
yearly_medians_list = yearly_medians.toList(yearly_medians.size())

# Get the number of images.
num_images = yearly_medians_list.size().getInfo()

# Loop over the list and export each image.
for i in range(num_images):
    year = years.get(i).getInfo()
    task = ee.batch.Export.image.toDrive(
        image=ee.Image(yearly_medians_list.get(i)),
        description = 'Planet_Satellite_5m_median_' + str(year),
        fileNamePrefix = 'Planet_Satellite_5m_median_' + str(year),
        region=piura_shp,
        scale=5,
        maxPixels=1e13
    )
    task.start()

# vis_basemap = {'bands': ['R_median', 'G_median', 'B_median'], 'min': 64, 'max': 5454, 'gamma': 1.8}
# Map = geemap.Map()
# Map.centerObject(piura_shp, 13)
# Map.addLayer(yearly_medians.first(), vis_basemap, 'last_image')
# Map

# Copernicus Land Cover
100m resolution
2015-01-01 to 2019-12-31

https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global#description

In [ ]:
dataset = ee.Image('COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019')
.select('discrete_classification')

# Landsat
Landsat provides 30m resolution from 1985 to today.

An annual composite is made to correct for cloud coverage, with instructions from the following video: https://www.youtube.com/watch?v=GzQ9HSXz9vE


In [286]:
L5coll = (ee.ImageCollection('LANDSAT/LT05/C01/T1_SR')
          .filter(ee.Filter.lt('CLOUD_COVER',25))
          .select(['B3', 'B2', 'B1'])
          .filterBounds(piura_shp))

def blend_image(image):
    filled1a = image.focal_mean(2, 'square', 'pixels', 1)
    return filled1a.blend(image)

L7coll = (ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')
          .filter(ee.Filter.lt('CLOUD_COVER',25))
          .select(['B3', 'B2', 'B1'])
          .filterBounds(piura_shp)
          .map(blend_image))

def rename_bands(image):
    return image.rename(['B0', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11'])

L8coll = (ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
          .filter(ee.Filter.lt('CLOUD_COVER',5))
          .filterBounds(piura_shp)
          .map(rename_bands)
          .select(['B3', 'B2', 'B1']))


years = ee.List.sequence(1984, 2024)

landsat_basemap = ee.ImageCollection(L5coll.merge(L7coll.merge(L8coll)))
# # Function to calculate geometric median for each year.
# def calculate_median(year):
#     start_date = ee.Date.fromYMD(year, 1, 1)
#     end_date = ee.Date.fromYMD(year, 12, 31)
#     yearly_collection = landsat_basemap.filterDate(start_date, end_date)
#     median = yearly_collection.reduce(ee.Reducer.mean())
#     return median.set('year', year)

# # Map over the list of years and calculate geometric medians.
# yearly_medians = ee.ImageCollection(years.map(calculate_median)).map(lambda image: image.clip(piura_shp))
# yearly_medians

In [298]:
# Define the date range
start_date = '1985-01-01'
end_date = '2024-03-03'

L5coll = (ee.ImageCollection('LANDSAT/LT05/C01/T1_SR')
          .filter(ee.Filter.lt('CLOUD_COVER',25))
          .filterDate(start_date, end_date)
          .select(['B3', 'B2', 'B1'])
          .filterBounds(piura_shp))

def blend_image(image):
    filled1a = image.focal_mean(2, 'square', 'pixels', 1)
    return filled1a.blend(image)

L7coll = (ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')
          .filterDate(start_date, end_date)
          .select(['B3', 'B2', 'B1'])
          .filterBounds(piura_shp)
          .map(blend_image))

def rename_bands(image):
    return image.rename(['B0', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11'])

L8coll = (ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
          .filterDate(start_date, end_date)
          .filterBounds(piura_shp)
          .map(rename_bands)
          .select(['B3', 'B2', 'B1']))

def mask_l8_sr(image):
    # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloud_shadow_bit_mask = (1 << 3)
    clouds_bit_mask = (1 << 5)
    # Get the pixel QA band.
    qa = image.select('pixel_qa')
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0) and (qa.bitwiseAnd(clouds_bit_mask).eq(0))
    return image.updateMask(mask)

SATELLITE_SR = 'LANDSAT/LC08/C01/T1_SR'
RGB = ['B4', 'B3', 'B2']
dataset = ee.ImageCollection(SATELLITE_SR).filterBounds(piura_shp).filterDate(start_date, end_date).map(
    mask_l8_sr).select(RGB)

# Function to calculate geometric median for each year.
def calculate_median(year):
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = ee.Date.fromYMD(year, 12, 31)
    yearly_collection = dataset.filterDate(start_date, end_date)
    median = yearly_collection.reduce(ee.Reducer.median())
    return median.set('year', year)

# Map over the list of years and calculate geometric medians.
yearly_medians = ee.ImageCollection(years.map(calculate_median))


# Dynamic World
10m land use, from 2015-06-27 to today. Near real time.


https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_DYNAMICWORLD_V1#colab-python


# ESA WorldCover
10m resolution
2020-01-01 - 2022-12-31

https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v100
https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v200

In [316]:
dataset = ee.ImageCollection('ESA/WorldCover/v100').map(lambda image: image.clip(piura_shp))
dataset

# visualization = {
#     'bands': ['Map'],
# }

# m = geemap.Map()
# m.center_object(dataset)
# m.add_layer(dataset, visualization, 'Landcover')
# m

Name,Description
Map,Landcover class


# ESRI Land Use Land Cover

https://livingatlas.arcgis.com/landcover/

In [ ]:
esri_lulc2020 = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m")


# ESRI Topography

All of these should have a finely traced but properly scaled grid with tick marks corresponding to decimal long./lat. degrees.

# NASA Elevation

In [128]:
# Import the dataset and select the elevation band.
dataset = ee.Image('NASA/NASADEM_HGT/001')
elevation = dataset.select('elevation')

# Add a white background image to the map.
background = ee.Image(1)
background_vis = {'min': 0, 'max': 1}
background_layer = background.visualize(**background_vis)

# Set elevation visualization properties.
elevation_vis = {'min': 0, 'max': 2000}

# Set elevation <= 0 as transparent.
elevation_masked = elevation.updateMask(elevation.gt(0)).clip(piura_shp)
elevation_masked
Map = geemap.Map()
Map.addLayer(background_layer, {}, 'Background')
Map.addLayer(elevation_masked, elevation_vis, 'Elevation')
Map.centerObject(piura_shp, 12)
Map

Map(center=[8.517382416611035, -81.00936813268046], controls=(WidgetControl(options=['position', 'transparent_…

# Hydrology

https://developers.google.com/earth-engine/datasets/catalog/MERIT_Hydro_v1_0_1

In [132]:
dataset = ee.Image('MERIT/Hydro/v1_0_1').clip(piura_shp)

visualization = {'bands': ['viswth']}

Map.centerObject(piura_shp, 12)

Map.addLayer(dataset, visualization, 'River width')
Map

Map(bottom=499687.0, center=[8.517382416611035, -81.00936813268046], controls=(WidgetControl(options=['positio…